참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 07.005<br>ex07.005

부정정보: 모멘트 면적법<br>Statically Indeterminate Beam : Moment Area Method

p. 306

### 문제에서 주어진 변수<br>Given Parameters

#### 보의 길이<br>Length of the beam

In [ ]:
L_AB_m = sy.symbols('L_{AB}[m]', real=True, nonnegative=True)

s_d = {
    L_AB_m: 10,
}


#### 하중<br>Load

In [ ]:
w0_N_m = sy.symbols('w0[N/m]', real=True)

s_d.update(
    {
        w0_N_m: -1,
    }
)


#### 재료와 단면 특성<br>Material & section properties

In [ ]:
E_Pa, I_m4 = sy.symbols('E[Pa], I[m^{4}]', positive=True)

s_d.update(
    {
        E_Pa: 200e9,
        I_m4: 20e6 * (1e-3) ** 4,
    }
)


#### 자유물체도<br>Free body diagram

In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

points_list = [
    {'x_m': 0, 'text':'A'},
    {'x_m': L_AB_m.subs(s_d), 'text':'B'},
]

reaction_list = [
    {'x_m': 0},
    {'x_m': L_AB_m.subs(s_d)},
]

v_load_list = []

dist_load_list = [
    {'x_begin_m': 0, 'x_end_m': L_AB_m.subs(s_d), 'text': 'w0'},
]

moment_list = [
    {'x_m': L_AB_m.subs(s_d), 'direction': 'cw', 'text': 'M', 'open':'left'},
]

draw_diagrams.draw_beam(L_AB_m.subs(s_d), points_list, reaction_list, v_load_list=v_load_list, dist_load_list=dist_load_list, moment_list=moment_list)

In [ ]:
x_m = sy.symbols('x[m]', nonnegative=True)
x_A_m = 0
x_B_m = L_AB_m


### 아직 알지 못하는 반력<br>Reaction forces unknown yet

In [ ]:
R_A_N, M_B_Nm, R_B_N = sy.symbols('R_{A}[N], M_{B}[Nm], R_{B}[N]', real=True)

### 모멘트면적법<br>Moment area method

#### 분포하중 $w_0$에 의한 굽힘모멘트선도<br>Bending moment diagram by distributed load $w_0$

p.279 표 6.2의 분포하중 $w_0$에 의한 처짐을 두번 미분해본다.<br>Differentiate the deflection by distributed load $w_0$ of p.279 Table 6.2

$$
\delta_{w_0} = \frac{w_0 x^2}{24EI} (6L^2 - 4L x + x^2)
$$

In [ ]:
delta_w0_m = (w0_N_m * x_m ** 2 / (24 * E_Pa * I_m4)) * (6 * L_AB_m ** 2 - 4 * L_AB_m * x_m + x_m ** 2)

고정단의 위치가 반대이므로 $x$를 $L-x$로 바꾼다.<br>Because the fixed support location is the opposite,replace $x$ with $L-x$.

In [ ]:
delta_w0_m = delta_w0_m.subs(x_m, L_AB_m - x_m)

두번 미분한다<br>Differentiate twice

In [ ]:
BMD_w0 = sy.simplify(E_Pa * I_m4 * sy.diff(delta_w0_m, x_m, x_m))

In [ ]:
BMD_w0

굽힘모멘트선도<br>Bending moment diagram

In [ ]:
splot.plot(
    BMD_w0.subs(s_d), (x_m, 0, L_AB_m.subs(s_d)), 
    xlabel='$10\\frac{x[m]}{L[m]}$',
    ylabel='$\\frac{BM[Nm]}{w_0 [N/m]}$'
)

p. 267 표 6.1에서 $n=2$라면 면적 $A$와 도심의 위치 $\overline{x}$ 는 다음과 같다.<br>From p. 267 Table 6.1, if $n=2$, area $A$ and centroid location $\overline{x}$ are as follows.

$$
\begin{align}
    A&=\frac{1}{3}bh \\
    \overline{x}&=\frac{3}{4}b\\
\end{align}
$$

면적<br>Area

In [ ]:
A_w0 = L_AB_m * BMD_w0.subs(x_m, L_AB_m) / 3

In [ ]:
A_w0

도심거리<br>Centroid distance

In [ ]:
x_bar_w0 = 3 * L_AB_m / 4

$B$점의 접선으로부터 $A$점의 처짐<br>Deflection at $A$ from the tangential line of $B$

In [ ]:
EI_delta_w0 = A_w0 * x_bar_w0

In [ ]:
EI_delta_w0

#### 외팔보 끝에 가해지는 반력 $R$에 의한 굽힘모멘트 선도<br>Bending moment diagram by reaction force $R$ at the end of the cantilever

p. 279 Table 6.2

$$
BMD_{R_A} = R_A x
$$

In [ ]:
BMD_RA = sy.simplify(R_A_N * x_m)

In [ ]:
BMD_RA

p. 267 표 6.1에서 $n=1$라면 면적 $A$와 도심의 위치 $\overline{x}$ 는 다음과 같다.<br>From p. 267 Table 6.1, if $n=1$, area $A$ and centroid location $\overline{x}$ are as follows.

$$
\begin{align}
    A&=\frac{1}{2}bh \\
    \overline{x}&=\frac{2}{3}b\\
\end{align}
$$

면적<br>Area

In [ ]:
A_RA = L_AB_m * R_A_N * L_AB_m / 2

In [ ]:
A_RA

도심거리<br>Centroid distance

In [ ]:
x_bar_RA = 2 * L_AB_m / 3

$B$점의 접선으로부터 $A$점의 처짐<br>Deflection at $A$ from the tangential line of $B$

In [ ]:
EI_delta_RA = A_RA * x_bar_RA

In [ ]:
EI_delta_RA

#### 두 처짐을 중첩함<br>Superpose two $\delta$'s

In [ ]:
super_eq = sy.Eq(EI_delta_w0 + EI_delta_RA, 0)

In [ ]:
super_eq

#### $R$에 관하여 풂<br>Solve for $R$

In [ ]:
R_A_N_sol = sy.solve(super_eq, R_A_N)[0]

In [ ]:
R_A_N_sol

### 평형방정식<br>Equilibrium equation

수직방향<br>Vertical direction

$$
R_A + R_B + w_0 L = 0
$$

In [ ]:
fy_eq = sy.Eq(R_A_N + R_B_N + w0_N_m * L_AB_m)

In [ ]:
fy_eq

In [ ]:
fy_eq.subs(R_A_N, R_A_N_sol)

#### B 점에서의 수직 반력<br>Vertical reaction force at B

In [ ]:
R_B_N_sol = sy.solve(fy_eq.subs(R_A_N, R_A_N_sol), R_B_N)

In [ ]:
R_B_N_sol

$B$점 중심 회전방향<br>Rotation direction around $B$

$$
M_B + R_A \cdot L + (w_0 L) \cdot \frac{1}{2}L=0
$$

In [ ]:
m_eq = sy.Eq(+ M_B_Nm + R_A_N * L_AB_m + sy.Rational(1, 2) * w0_N_m * L_AB_m ** 2)

In [ ]:
m_eq

In [ ]:
sy.simplify(m_eq.subs(R_A_N, R_A_N_sol))

#### B 점에서의 모멘트반력<br>Reaction moment at B

In [ ]:
M_B_Nm_sol = sy.solve(m_eq.subs(R_A_N, R_A_N_sol), M_B_Nm)[0]

In [ ]:
sy.simplify(M_B_Nm_sol)